In [1]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    JSONAgentOutputParser,
    ReActSingleInputOutputParser,
)
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun, tool
from langchain.tools.render import render_text_description_and_args, format_tool_to_openai_function
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder
from langchain.schema import ChatMessage, SystemMessage

from pymatgen.core.periodic_table import _pt_data

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever
)
from llamp.arxiv.agents import ArxivAgent

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)

# OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
OPENAI_GPT_MODEL = "gpt-4"

In [2]:
import re

mp_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo-1106",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    max_retries=5,
    # streaming=True
)

llm = ChatOpenAI(
    temperature=0.7,
    model="gpt-4",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    # streaming=True
)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = [
    MPThermoExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElasticityExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPDielectricExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPMagnetismExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElectronicExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPSummaryExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPSynthesisExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPStructureRetriever(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    # ArxivAgent(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    # arxiv,
    # wikipedia,
]

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages[0].prompt.template = re.sub(
    r"\s+", " ",
    """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    user to clarify their queries if needed. Please note that you don't have 
    direct control over MP but through multiple assistant agents to help you. 
    You need to provide complete context in the input for them to do their job.
    """).replace("\n", " ") + prompt.messages[0].prompt.template

prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm.bind(stop=["Observation"])
    # | map_reduce_chain  # TODO: Add map-reduce after LLM
    | JSONAgentOutputParser()
)


conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

agent_kwargs = {
    "handle_parsing_errors": True,
    "extra_prompt_messages": [
        MessagesPlaceholder(variable_name="chat_history"),
        # SystemMessage(content=re.sub(
        #     r"\s+", " ",
        #     """You are a helpful data-aware agent that can consult materials-related
        #     data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
        #     user to clarify their queries if needed. Please note that you don't have 
        #     direct control to MP but through multiple assistant agents to help you. 
        #     You need to provide complete context for them to do their job. 
        #     """).replace("\n", " ")
        # )
        ],
    "early_stopping_method": 'generate',
}

agent_executor = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=20,
    memory=conversational_memory,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
)

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [3]:
agent_executor.invoke({
    "input": "Could you insert one Li atom at the tetrahedral interstitial site in diamond cubic Si? Pull out the most stable DC Si structure from MP directly in JSON string, modify the retrived structure directly, and give me the final pymatgen structure as a JSON string.",
})



> Entering new AgentExecutor chain...


Thought: The user is asking to insert a Li atom at the tetrahedral interstitial site in a diamond cubic Silicon (Si) structure. First, I need to retrieve the most stable diamond cubic (DC) Si structure from the Materials Project. 

Action:
```
{
  "action": "MPStructureRetriever",
  "action_input": "I need the most stable DC Si structure"
}
```

> Entering new AgentExecutor chain...


/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Action:
```json
{
  "action": "search_materials_structure__get",
  "action_input": {
    "formula": "Si",
    "is_stable": true,
    "sort_fields": "-efermi",
    "limit": 1,
    "fields": "material_id,formula_pretty,structure"
  }
}
```{"formula": "Si", "is_stable": true, "sort_fields": "-efermi", "limit": 1, "fields": "material_id,formula_pretty,structure"}


Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Question: How to insert one Li atom at the tetrahedral interstitial site in the retrieved DC Si structure?
Thought: I can use the "search_materials_structure__get" tool to retrieve the DC Si structure and then modify it to insert a Li atom at the tetrahedral interstitial site.
Action:
```json
{
  "action": "search_materials_structure__get",
  "action_input": {
    "formula": "Si",
    "spacegroup_number": "227",
    "fields": "structure"
  }
}
```
{"formula": "Si", "spacegroup_number": "227", "fields": "structure"}


Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

[{'material_id': 'mp-149', 'structure': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[3.333573, 0.0, 1.924639], [1.111191, 3.142924, 1.924639], [0.0, 0.0, 3.849278]], 'pbc': [True, True, True], 'a': 3.8492784033699095, 'b': 3.8492794116013456, 'c': 3.849278, 'alpha': 60.00001213094421, 'beta': 60.00000346645984, 'gamma': 60.00001097545789, 'volume': 40.32952684741405}, 'properties': {}, 'sites': [{'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.875, 0.875, 0.875], 'xyz': [3.8891685, 2.7500584999999997, 6.7362365], 'properties': {'magmom': -0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.125, 0.125, 0.125], 'xyz': [0.5555955, 0.3928655, 0.9623195], 'properties': {'magmom': -0.0}, 'label': 'Si'}]}}, {'material_id': 'mp-16220', 'structure': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0.0, 'lattice': {'matrix': [[0.0, 7.33856771, 7.33856771], [7.33856771, -0.0, 7.33856771], [7.

{'input': 'Could you insert one Li atom at the tetrahedral interstitial site in diamond cubic Si? Pull out the most stable DC Si structure from MP directly in JSON string, modify the retrived structure directly, and give me the final pymatgen structure as a JSON string.',
 'chat_history': [],
 'output': "The final structure after inserting a Li atom at the tetrahedral interstitial site in the diamond cubic Si structure is as follows: {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[3.333573, 0.0, 1.924639], [1.111191, 3.142924, 1.924639], [0.0, 0.0, 3.849278]], 'pbc': [True, True, True], 'a': 3.8492784033699095, 'b': 3.8492794116013456, 'c': 3.849278, 'alpha': 60.00001213094421, 'beta': 60.00000346645984, 'gamma': 60.00001097545789, 'volume': 40.32952684741405}, 'properties': {}, 'sites': [{'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.875, 0.875, 0.875], 'xyz': [3.8891685, 2.7500584999999997, 6.7362365], 'properties': {'magmom':

In [4]:
print(agent_executor.memory.chat_memory.messages[1].content)

The final structure after inserting a Li atom at the tetrahedral interstitial site in the diamond cubic Si structure is as follows: {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[3.333573, 0.0, 1.924639], [1.111191, 3.142924, 1.924639], [0.0, 0.0, 3.849278]], 'pbc': [True, True, True], 'a': 3.8492784033699095, 'b': 3.8492794116013456, 'c': 3.849278, 'alpha': 60.00001213094421, 'beta': 60.00000346645984, 'gamma': 60.00001097545789, 'volume': 40.32952684741405}, 'properties': {}, 'sites': [{'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.875, 0.875, 0.875], 'xyz': [3.8891685, 2.7500584999999997, 6.7362365], 'properties': {'magmom': -0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.125, 0.125, 0.125], 'xyz': [0.5555955, 0.3928655, 0.9623195], 'properties': {'magmom': -0.0}, 'label': 'Si'}, {'species': [{'element': 'Li', 'occu': 1}], 'abc': [0.5, 0.5, 0.5], 'xyz': [2.222382, 1.571462, 1.924639], 'properties

In [5]:

llm_gpt = ChatOpenAI(
    temperature=0.7,
    model="gpt-3.5-turbo-1106",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    # streaming=True
)

response = llm_gpt.invoke(agent_executor.memory.chat_memory.messages[0].content)


In [12]:
response.content

'Here\'s the JSON string for the most stable diamond cubic Si structure retrieved directly from MP:\n\n```json\n{\n  "sites": [\n    {\n      "species": [\n        {\n          "element": "Si",\n          "occu": 1\n        }\n      ],\n      "abc": [\n        0,\n        0,\n        0\n      ],\n      "xyz": [\n        0,\n        0,\n        0\n      ],\n      "label": "Si",\n      "properties": {}\n    },\n    {\n      "species": [\n        {\n          "element": "Si",\n          "occu": 1\n        }\n      ],\n      "abc": [\n        0.25,\n        0.25,\n        0.25\n      ],\n      "xyz": [\n        1.780000e-01,\n        1.780000e-01,\n        1.780000e-01\n      ],\n      "label": "Si",\n      "properties": {}\n    }\n  ],\n  "lattice": {\n    "matrix": [\n      [\n        3.866002,\n        0,\n        2.243049\n      ],\n      [\n        3.866002,\n        3.866002,\n        0\n      ],\n      [\n        0,\n        3.866002,\n        2.243049\n      ]\n    ],\n    "a": 5.4

In [6]:
print(response.content)

Here's the JSON string for the most stable diamond cubic Si structure retrieved directly from MP:

```json
{
  "sites": [
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [
        0,
        0,
        0
      ],
      "xyz": [
        0,
        0,
        0
      ],
      "label": "Si",
      "properties": {}
    },
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [
        0.25,
        0.25,
        0.25
      ],
      "xyz": [
        1.780000e-01,
        1.780000e-01,
        1.780000e-01
      ],
      "label": "Si",
      "properties": {}
    }
  ],
  "lattice": {
    "matrix": [
      [
        3.866002,
        0,
        2.243049
      ],
      [
        3.866002,
        3.866002,
        0
      ],
      [
        0,
        3.866002,
        2.243049
      ]
    ],
    "a": 5.431,
    "b": 5.431,
    "c": 5.431,
    "alpha": 109.471219,
    "

In [14]:
{
  "sites": [
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [
        0,
        0,
        0
      ],
      "xyz": [
        0,
        0,
        0
      ],
      "label": "Si",
      "properties": {}
    },
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [
        0.25,
        0.25,
        0.25
      ],
      "xyz": [
        1.780000e-01,
        1.780000e-01,
        1.780000e-01
      ],
      "label": "Si",
      "properties": {}
    }
  ],
  "lattice": {
    "matrix": [
      [
        3.866002,
        0,
        2.243049
      ],
      [
        3.866002,
        3.866002,
        0
      ],
      [
        0,
        3.866002,
        2.243049
      ]
    ],
    "a": 5.431,
    "b": 5.431,
    "c": 5.431,
    "alpha": 109.471219,
    "beta": 109.471219,
    "gamma": 109.471219,
    "volume": 129.584354898
  },
  "properties": {
    "space_group": {
      "number": 227,
      "source": "spglib",
      "symbol": "Fd-3m"
    }
  }
}

{'sites': [{'species': [{'element': 'Si', 'occu': 1}],
   'abc': [0, 0, 0],
   'xyz': [0, 0, 0],
   'label': 'Si',
   'properties': {}},
  {'species': [{'element': 'Si', 'occu': 1}],
   'abc': [0.25, 0.25, 0.25],
   'xyz': [0.178, 0.178, 0.178],
   'label': 'Si',
   'properties': {}}],
 'lattice': {'matrix': [[3.866002, 0, 2.243049],
   [3.866002, 3.866002, 0],
   [0, 3.866002, 2.243049]],
  'a': 5.431,
  'b': 5.431,
  'c': 5.431,
  'alpha': 109.471219,
  'beta': 109.471219,
  'gamma': 109.471219,
  'volume': 129.584354898},
 'properties': {'space_group': {'number': 227,
   'source': 'spglib',
   'symbol': 'Fd-3m'}}}

In [1]:
from pymatgen.core import Structure

structure = Structure.from_dict(
    {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[3.333573, 0.0, 1.924639], [1.111191, 3.142924, 1.924639], [0.0, 0.0, 3.849278]], 'pbc': [True, True, True], 'a': 3.8492784033699095, 'b': 3.8492794116013456, 'c': 3.849278, 'alpha': 60.00001213094421, 'beta': 60.00000346645984, 'gamma': 60.00001097545789, 'volume': 40.32952684741405}, 'properties': {}, 'sites': [{'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.875, 0.875, 0.875], 'xyz': [3.8891685, 2.7500584999999997, 6.7362365], 'properties': {'magmom': -0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.125, 0.125, 0.125], 'xyz': [0.5555955, 0.3928655, 0.9623195], 'properties': {'magmom': -0.0}, 'label': 'Si'}, {'species': [{'element': 'Li', 'occu': 1}], 'abc': [0.5, 0.5, 0.5], 'xyz': [2.222382, 1.571462, 1.924639], 'properties': {}, 'label': 'Li'}]}
)

structure.to(filename="./05-crystal-Si-Li-interstitial.cif", fmt="cif")
structure.to_ase_atoms().write("./05-crystal-Si-Li-interstitial.xyz")

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/pymatgen/core/structure.py:990: UserWarning: Not all sites have property magmom. Missing values are set to None.
  warnings.warn(f"Not all sites have property {key}. Missing values are set to None.")


In [4]:
structure = Structure.from_dict(
    {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[3.333573, 0.0, 1.924639], [1.111191, 3.142924, 1.924639], [0.0, 0.0, 3.849278]], 'pbc': [True, True, True], 'a': 3.8492784033699095, 'b': 3.8492794116013456, 'c': 3.849278, 'alpha': 60.00001213094421, 'beta': 60.00000346645984, 'gamma': 60.00001097545789, 'volume': 40.32952684741405}, 'properties': {}, 'sites': [{'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.875, 0.875, 0.875], 'xyz': [3.8891685, 2.7500584999999997, 6.7362365], 'properties': {'magmom': -0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.125, 0.125, 0.125], 'xyz': [0.5555955, 0.3928655, 0.9623195], 'properties': {'magmom': -0.0}, 'label': 'Si'}]}
)

structure.to(filename="./05-crystal-Si.cif", fmt="cif")
structure.to_ase_atoms().write("./05-crystal-Si.xyz")

In [3]:

structure = Structure.from_dict(
    {
  "sites": [
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [0, 0, 0],
      "xyz": [0, 0, 0],
      "label": "Si",
      "properties": {}
    },
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [0.25, 0.25, 0.25],
      "xyz": [0.178, 0.178, 0.178],
      "label": "Si",
      "properties": {}
    },
    {
      "species": [
        {
          "element": "Li",
          "occu": 1
        }
      ],
      "abc": [0.5, 0.5, 0.5],
      "xyz": [0.5, 0.5, 0.5],
      "label": "Li",
      "properties": {}
    }
  ],
  "lattice": {
    "matrix": [
      [3.866002, 0, 2.243049],
      [3.866002, 3.866002, 0],
      [0, 3.866002, 2.243049]
    ],
    "a": 5.431,
    "b": 5.431,
    "c": 5.431,
    "alpha": 109.471219,
    "beta": 109.471219,
    "gamma": 109.471219,
    "volume": 129.584354898
  },
  "properties": {
    "space_group": {
      "number": 227,
      "source": "spglib",
      "symbol": "Fd-3m"
    }
  }
}
)

structure.to(filename="./05-crystal-Si-Li-interstitial-gpt3.5.cif", fmt="cif")
structure.to_ase_atoms().write("./05-crystal-Si-Li-interstitial-gpt3.5.xyz")